# Hidden Layer

In [1]:
from abc import abstractmethod, ABC
import numpy as np

In [2]:
class Tensor:

    def __init__(self, data):
        self.data = np.array(data)

        self.grad = None
        self.gradient_fn = lambda: None
        self.parents = set()

    def gradient(self):
        if self.gradient_fn:
            self.gradient_fn()

        for p in self.parents:
            p.gradient()

    def size(self):
        return self.data.shape[-1]

In [3]:
class DataLoader:

    def __init__(self, batch_size):
        self.batch_size = batch_size

        self.features = [[28.1, 58.0],
                         [22.5, 72.0],
                         [31.4, 45.0],
                         [19.8, 85.0],
                         [27.6, 63]]
        self.labels = [[165],
                       [95],
                       [210],
                       [70],
                       [155]]

    def __len__(self):  # 3
        return len(self.features)

    def __getitem__(self, index):  # 4
        return (Tensor(self.features[index: index + self.batch_size]),
                Tensor(self.labels[index: index + self.batch_size]))

In [4]:
class Layer(ABC):

    def __call__(self, x: Tensor):
        return self.forward(x)

    @abstractmethod
    def forward(self, x: Tensor):
        pass

    def parameters(self):
        return []

In [5]:
np.random.seed(99)

class Linear(Layer):

    def __init__(self, in_size, out_size):
        self.in_size = in_size
        self.out_size = out_size

        self.weight = Tensor(np.random.rand(out_size, in_size) / in_size)
        self.bias = Tensor(np.zeros(out_size))

    def forward(self, x: Tensor):
        p = Tensor(x.data @ self.weight.data.T + self.bias.data)

        def gradient_fn():
            self.weight.grad = p.grad.T @ x.data / len(x.data)
            self.bias.grad = np.sum(p.grad, axis=0) / len(x.data)
            x.grad = p.grad @ self.weight.data

        p.gradient_fn = gradient_fn
        p.parents = {self.weight, self.bias, x}
        return p

    def parameters(self):
        return [self.weight, self.bias]

In [6]:
class Sequential(Layer):

    def __init__(self, layers):
        self.layers = layers

    def forward(self, x: Tensor):
        for l in self.layers:
            x = l(x)
        return x

    def parameters(self):
        return [p for l in self.layers for p in l.parameters()]

In [7]:
class MSELoss:

    def __call__(self, p: Tensor, y: Tensor):
        mse = Tensor(((p.data - y.data) ** 2).mean())

        def gradient_fn():
            p.grad = (p.data - y.data) * 2

        mse.gradient_fn = gradient_fn
        mse.parents = {p}
        return mse

In [8]:
class SGD:

    def __init__(self, parameters, lr):
        self.parameters = parameters
        self.lr = lr

    def backward(self):
        for p in self.parameters:
            p.data -= p.grad * self.lr

In [9]:
LEARNING_RATE = 0.00001
BATCHES = 2
EPOCHS = 100

In [10]:
dataset = DataLoader(BATCHES)

feature, label = dataset[0]
model = Sequential([Linear(feature.size(), 4),
                    Linear(4, label.size())])
loss = MSELoss()
sgd = SGD(model.parameters(), LEARNING_RATE)

for epoch in range(EPOCHS):
    for i in range(0, len(dataset), dataset.batch_size):
        feature, label = dataset[i]

        prediction = model(feature)
        error = loss(prediction, label)

        error.gradient()
        sgd.backward()

print(f'prediction: {prediction.data}')
print(f'error: {error.data}')
print(f"hidden weight: {model.layers[0].weight.data}")
print(f"hidden bias: {model.layers[0].bias.data}")
print(f"output weight: {model.layers[1].weight.data}")
print(f"output bias: {model.layers[1].bias.data}")

prediction: [[156.72406292]]
error: 2.97239296238889
hidden weight: [[ 1.76093526 -0.17343877]
 [ 1.02638991 -0.30940862]
 [ 1.83729626 -0.18342356]
 [ 0.86270593 -0.15644664]]
hidden bias: [0.0275739  0.01086594 0.02741749 0.01401818]
output weight: [[1.7206489  0.9828031  1.7754612  0.87087029]]
output bias: [0.03443334]
